In [1]:
import numpy as np
import scipy.io as scio
import scipy.ndimage as im
import imageio
import matplotlib.pyplot as plt
import keras

from keras import models
from keras import layers
from keras import optimizers
from keras import applications
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model, Model
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Dense, Activation, Flatten, Conv2D, RepeatVector
from keras.layers import GlobalAveragePooling2D, BatchNormalization, ZeroPadding2D, UpSampling2D
from keras.models import Sequential
from keras.layers import Reshape, Add, Multiply, Lambda, AveragePooling2D
from keras.layers import concatenate
from keras.layers import MaxPooling2D, Dropout, Input, MaxPool2D
from keras.optimizers import SGD, Adam, Nadam, RMSprop, Adagrad
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import PReLU
from keras.activations import linear as linear_activation
from keras import initializers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import fbeta_score
from sklearn.metrics import classification_report,confusion_matrix
from  sklearn.metrics  import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
!git clone https://github.com/nilesh97/Multilabel-LandCover-Classification.git

Cloning into 'Multilabel-LandCover-Classification'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 5230 (delta 0), reused 32 (delta 0), pack-reused 5197
Receiving objects: 100% (5230/5230), 984.98 MiB | 45.02 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (8106/8106), done.


In [3]:
# Load the labels
path_to_labels = 'Multilabel-LandCover-Classification/UCMerced_LandUse_Multilabels/LandUse_multilabels.mat' # CHANGE THIS PATH ACCORDINGLY

mat_labels   = scio.loadmat(path_to_labels)
numpy_labels = np.transpose( mat_labels['labels'] )
numpy_labels = np.array(numpy_labels, dtype=np.float32)

print( "The shape of the label matrix is {}.\n".format(numpy_labels.shape) )


# Load the images.
path_to_images = 'Multilabel-LandCover-Classification/UCMerced_LandUse_Images/'  # CHANGE THIS PATH ACCORDINGLY

image_types = [ 'agricultural',  'airplane',    'baseballdiamond', 'beach',   'buildings',          'chaparral',         'denseresidential',
                'forest',        'freeway',     'golfcourse',      'harbor',  'intersection',       'mediumresidential', 'mobilehomepark',
                'overpass',      'parkinglot',  'river',           'runway',  'sparseresidential',  'storagetanks',      'tenniscourt' ]

extension = '.tif'


images = []

# In reality not all images are the same size. So we will crop them in the smallest dimensions possible.
x_limit = 256
y_limit = 256


for current_type in image_types:

    for index in range(100):

        str_index = str(index)

        if index < 10:
            str_index = '0' + str_index

        current_image = imageio.imread(path_to_images + current_type + '/' + current_type + str_index + extension)

        # Crop the images.
        if current_image.shape[0] > 256:
          remove_from_x = current_image.shape[0] - x_limit
          remove_from_y = current_image.shape[1] - y_limit
          current_image = current_image[remove_from_x:, remove_from_y:]
        
        if current_image.shape[0] != 256:
          pad_x = 256 - current_image.shape[0]
          z = np.zeros((pad_x,current_image.shape[1],3), dtype=np.int64)
          current_image = np.concatenate((current_image,z),axis=0)
        
        if current_image.shape[1] != 256:
          pad_y = 256 - current_image.shape[1]
          p = np.zeros((256,pad_y,3), dtype=np.int64)
          current_image = np.concatenate((current_image,p),axis=1)
          
        images.append(current_image)


print (len (images), len(images[0]), len (images[0][0]), len(images[0][0][0]))
images = np.array(images, dtype=np.float32)
images = images / 255
print("hi")

print( "The shape of the image samples matrix is {}.\n".format(images.shape) )


# Shuffle the data.
random_indices = np.arange( images.shape[0] )
np.random.seed(42)
np.random.shuffle(random_indices)

numpy_labels     = numpy_labels[random_indices]
images           = images[random_indices]


# Pickle the data.
np.save("UcmImages.npy", images)
np.save("UcmLabels.npy", numpy_labels)

The shape of the label matrix is (2100, 17).



/usr/local/lib/python3.6/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_packbits'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.6/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'decode_lzw'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.6/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'unpack_ints'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/usr/local/lib/python3.6/dist-packages/imageio/plugins/_tifffile.py:7285: UserWarning: module 'imageio.plugins._tifffile' has no attribute 'reverse_bitorder'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))


2100 256 256 3
hi
The shape of the image samples matrix is (2100, 256, 256, 3).



In [4]:
print(random_indices)

[1034 1176   67 ... 1130 1294  860]


In [0]:
#Split the data into training and test sets
OBSERVATIONS_FILE  = 'UcmImages.npy' # The file containing the data samples.
LABELS_FILE        = 'UcmLabels.npy' # The file containing the labels.
TESTING_DATA_NUM = 400

images = np.load(OBSERVATIONS_FILE)
labels = np.load(LABELS_FILE)

random_indices = np.arange( images.shape[0] )
np.random.seed(42)
np.random.shuffle(random_indices)

labels = labels[random_indices]
images = images[random_indices]

test_set    = images[:TESTING_DATA_NUM]
test_labels = labels[:TESTING_DATA_NUM]

train_set = images[TESTING_DATA_NUM:]
train_labels = labels[TESTING_DATA_NUM:]

In [18]:
print(random_indices)

[1034 1176   67 ... 1130 1294  860]


In [0]:
#Parameters For Data Augmentation later
ROTATION_RANGE  = 45
SHIFT_FRACTION  = 0.2
SHEAR_RANGE     = 0.0
ZOOM_RANGE      = 0.0
HORIZONTAL_FLIP = True
VERTICAL_FILP   = True

data_generator = ImageDataGenerator( 
                                         rotation_range=ROTATION_RANGE, width_shift_range=SHIFT_FRACTION, height_shift_range=SHIFT_FRACTION,
                                         shear_range=SHEAR_RANGE,       zoom_range=ZOOM_RANGE,            horizontal_flip=HORIZONTAL_FLIP,
                                         vertical_flip=VERTICAL_FILP)
data_generator.fit(train_set)

In [0]:
#Useful Callbacks:
def CALLBACKS():
  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, min_lr=10e-8, epsilon=0.01, verbose=1)
  early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
  callbacks= [lr_reducer, early_stopper]
  
  return callbacks

In [0]:
#Accuracies:
def findMetrics(yTrue, yPred):
  # precision overall
  positive_predictions = np.count_nonzero(yPred) # denominator
  true_positives       = np.sum( np.logical_and(yTrue == 1, yPred == 1) ) # numerator

  if positive_predictions == 0:
      precision = 0
  else:
      precision = true_positives / positive_predictions


  # recall overall
  relevant_positives = np.count_nonzero(yTrue) # denominator

  recall = true_positives / relevant_positives


  # F Measure overall
  numerator   = precision * recall
  denominator = precision + recall

  if denominator == 0:
      f_measure = 0
  else:
      f_measure = (2 * numerator) / denominator
  
  # precision per row/column
  positive_predictions_row = np.count_nonzero(yPred, axis=1) # denominators
  positive_predictions_col = np.count_nonzero(yPred, axis=0) # denominators

  true_positives_row       = np.sum( np.logical_and(yTrue == 1, yPred == 1), axis=1 ) # numerators
  true_positives_col       = np.sum( np.logical_and(yTrue == 1, yPred == 1), axis=0 ) # numerators

  positive_predictions_row = positive_predictions_row.astype('float')
  positive_predictions_col = positive_predictions_col.astype('float')

  true_positives_row       = true_positives_row.astype('float')
  true_positives_col       = true_positives_col.astype('float')

  precision_per_row        = np.true_divide( true_positives_row, positive_predictions_row, out=np.zeros_like(true_positives_row), where=positive_predictions_row!=0 )
  precision_per_col        = np.true_divide( true_positives_col, positive_predictions_col, out=np.zeros_like(true_positives_col), where=positive_predictions_col!=0 )

  avrg_precision_row       = np.mean(precision_per_row)
  avrg_precision_col       = np.mean(precision_per_col)

  # multi_label accuracy overall
  accuracy2 = true_positives / ( np.sum( np.logical_or(yTrue == 1, yPred == 1) ) )
  
  acc2_denominator_row = np.sum( np.logical_or(yTrue == 1, yPred == 1), axis=1 )
  
  acc2_denominator_row = acc2_denominator_row.astype('float')
  
  accuracy2_row        = np.true_divide( true_positives_row, acc2_denominator_row, out=np.zeros_like(true_positives_row), where=acc2_denominator_row!=0 )
  
  avrg_acc2_row        = np.mean(accuracy2_row)
  
  # recall per row/column
  relevant_positives_row   = np.count_nonzero(yTrue, axis=1) # denominators
  relevant_positives_col   = np.count_nonzero(yTrue, axis=0) # denominators

  relevant_positives_row   = relevant_positives_row.astype('float')
  relevant_positives_col   = relevant_positives_col.astype('float')

  recall_per_row           = np.true_divide( true_positives_row, relevant_positives_row, out=np.zeros_like(true_positives_row), where=relevant_positives_row!=0 )
  recall_per_col           = np.true_divide( true_positives_col, relevant_positives_col, out=np.zeros_like(true_positives_col), where=relevant_positives_col!=0 )

  avrg_recall_row          = np.mean(recall_per_row)
  avrg_recall_col          = np.mean(recall_per_col)


  # F Measure per row
  numerator_row   = avrg_precision_row * avrg_recall_row
  denominator_row = avrg_precision_row + avrg_recall_row

  if denominator_row == 0:
      f1_measure_row = 0
      f2_measure_row = 0
  else:
      f1_measure_row = (2 * numerator_row) / denominator_row
      f2_measure_row = ((5 * numerator_row) / ((4 * avrg_precision_row) + (avrg_recall_row)))
      
  print("Accuracy is :: " + str(avrg_acc2_row))
  print("F1 Score is :: " + str(f1_measure_row))
  print("F2 Score is :: " + str(f2_measure_row))
  print("Precision row :: " + str(avrg_precision_row))
  print("Recall row :: " + str(avrg_recall_row))
  print("Precision column :: " + str(avrg_precision_col))
  print("Recall column :: " + str(avrg_recall_col))
  return accuracy2, precision, recall, f_measure, avrg_precision_row, avrg_recall_row, f1_measure_row, f2_measure_row, avrg_precision_col, avrg_recall_col, avrg_acc2_row

In [0]:
#different threshold values
def thresholding1(test_set, test_labels):
  out = model1.predict(test_set)
  out = np.array(out)
  #threshold = np.arange(0.1,0.9,0.05)
  #for t in threshold:
  for i in range(1):
    #Thresholding function
    threshold = np.arange(0.1,0.9,0.01)

    acc = []
    accuracies = []
    otsu = []

    best_threshold = np.zeros(out.shape[1])
    
    for i in range(out.shape[1]):
        y_prob = np.array(out[:,i])

        for j in threshold:
            y_pred = [1 if prob>=j else 0 for prob in y_prob]
            #acc.append( matthews_corrcoef(test_labels[:,i],y_pred))
            #acc.append( fbeta_score(test_labels[:,i],y_pred,beta=1))
            acc.append( accuracy_score(test_labels[:,i],y_pred))

        acc   = np.array(acc)
        index = np.where(acc==acc.max()) 
        accuracies.append(acc.max()) 
        best_threshold[i] = threshold[index[0][0]]
        acc = []
    
    #best_threshold=[0.45]*17
    print(best_threshold)

    y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(test_labels.shape[1])] for i in range(len(test_labels))])

    print(hamming_loss(test_labels,y_pred))

    x = findMetrics(test_labels, y_pred)
    print('  Classification Report:\n',classification_report(test_labels,y_pred),'\n')
    print(x)

#different threshold values
def thresholding2(test_set, test_labels):
  out = model2.predict(test_set)
  out = np.array(out)
  #threshold = np.arange(0.1,0.9,0.05)
  #for t in threshold:
  for i in range(1):
    #Thresholding function
    threshold = np.arange(0.1,0.9,0.01)

    acc = []
    accuracies = []
    otsu = []

    best_threshold = np.zeros(out.shape[1])
    
    for i in range(out.shape[1]):
        y_prob = np.array(out[:,i])

        for j in threshold:
            y_pred = [1 if prob>=j else 0 for prob in y_prob]
            #acc.append( matthews_corrcoef(test_labels[:,i],y_pred))
            #acc.append( fbeta_score(test_labels[:,i],y_pred,beta=1))
            acc.append( accuracy_score(test_labels[:,i],y_pred))

        acc   = np.array(acc)
        index = np.where(acc==acc.max()) 
        accuracies.append(acc.max()) 
        best_threshold[i] = threshold[index[0][0]]
        acc = []
    
    #best_threshold=[0.45]*17
    print(best_threshold)

    y_pred = np.array([[1 if out[i,j]>=best_threshold[j] else 0 for j in range(test_labels.shape[1])] for i in range(len(test_labels))])

    print(hamming_loss(test_labels,y_pred))

    x = findMetrics(test_labels, y_pred)
    print('  Classification Report:\n',classification_report(test_labels,y_pred),'\n')
    print(x)

In [0]:
def VGGNET():
  vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(247, 242, 3))
  
  for layers in vgg_model.layers:
    layers.trainable = True

  model = Sequential()

  model.add(vgg_model)
  #model.add(GlobalAveragePooling2D())
  model.add(Flatten(name='flatten_1'))

  model.add(Dense(17, activation='sigmoid', name='dense_1'))
  
  return model

def CA_VGG_LSTM():
  vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(247, 242, 3))

  model = Sequential()

  for layer in tuple(vgg_model.layers[:-5]):
    layer_type = type(layer).__name__
    model.add(layer)

  model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
  model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
  model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))

  model.add(Conv2D(17, kernel_size=(1, 1), strides=(1, 1), kernel_initializer='glorot_uniform'))
  model.add(Reshape((17, 28*28), input_shape=(28, 28, 17)))

  model.add(LSTM(17, input_shape=(17, 28*28), activation='tanh', kernel_initializer=initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=None)))
  
  model.add(Dense(17, activation='sigmoid'))

  return model

def CA_VGG_BILSTM():
  vgg_model = VGG16(include_top=False, weights='imagenet', input_shape=(247, 242, 3))

  model = Sequential()

  for layer in tuple(vgg_model.layers[:-5]):
    layer_type = type(layer).__name__
    model.add(layer)

  model.add(Conv2D(512, (3, 3), activation='relu', name='block5_conv1'))
  model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
  model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))

  model.add(Conv2D(17, kernel_size=(1, 1), strides=(1, 1), kernel_initializer='glorot_uniform'))
  model.add(Reshape((17, 28*28), input_shape=(28, 28, 17)))

  model.add(Bidirectional(LSTM(17, input_shape=(17, 28*28), activation='tanh', kernel_initializer=initializers.RandomUniform(minval=-0.1, maxval=0.1, seed=None)), merge_mode='sum'))
  
  model.add(Dense(17, activation='sigmoid'))

  return model

def GoogLeNet():
  base_model = InceptionV3(include_top=False, weights='imagenet', input_shape=(247, 242, 3))

  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation='relu')(x)
  # and a logistic layer -- let's say we have 17 classes
  predictions = Dense(17, activation='sigmoid')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)

  return model

def ResNet50():
  #base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(247, 242, 3))
  base_model = applications.resnet50.ResNet50(weights= 'imagenet', include_top=False, input_shape= (247,242,3))
  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation='relu')(x)
  # and a logistic layer -- let's say we have 17 classes
  predictions = Dense(17, activation='sigmoid')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)

  return model

def residual_block(input, input_channels=None, output_channels=None, kernel_size=(3, 3), stride=1):

    if output_channels is None:
        output_channels = input.get_shape()[-1].value
    if input_channels is None:
        input_channels = output_channels // 4

    strides = (stride, stride)

    x = BatchNormalization()(input)
    x = Activation('relu')(x)
    x = Conv2D(input_channels, (1, 1))(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(input_channels, kernel_size, padding='same', strides=stride)(x)

    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(output_channels, (1, 1), padding='same')(x)

    if input_channels != output_channels or stride != 1:
        input = Conv2D(output_channels, (1, 1), padding='same', strides=strides)(input)

    x = Add()([x, input])
    return x
  
def attention_block(input, input_channels=None, output_channels=None, encoder_depth=1):

    p = 1
    t = 2
    r = 1

    if input_channels is None:
        input_channels = input.get_shape()[-1].value
    if output_channels is None:
        output_channels = input_channels

    # First Residual Block
    for i in range(p):
        input = residual_block(input)

    # Trunc Branch
    output_trunk = input
    for i in range(t):
        output_trunk = residual_block(output_trunk)

    # Soft Mask Branch

    ## encoder
    ### first down sampling
    output_soft_mask = MaxPool2D(padding='same')(input)  # 32x32
    for i in range(r):
        output_soft_mask = residual_block(output_soft_mask)

    skip_connections = []
    for i in range(encoder_depth - 1):

        ## skip connections
        output_skip_connection = residual_block(output_soft_mask)
        skip_connections.append(output_skip_connection)
        # print ('skip shape:', output_skip_connection.get_shape())

        ## down sampling
        output_soft_mask = MaxPool2D(padding='same')(output_soft_mask)
        for _ in range(r):
            output_soft_mask = residual_block(output_soft_mask)

            ## decoder
    skip_connections = list(reversed(skip_connections))
    for i in range(encoder_depth - 1):
        ## upsampling
        for _ in range(r):
            output_soft_mask = residual_block(output_soft_mask)
        output_soft_mask = UpSampling2D()(output_soft_mask)
        ## skip connections
        output_soft_mask = Add()([output_soft_mask, skip_connections[i]])

    ### last upsampling
    for i in range(r):
        output_soft_mask = residual_block(output_soft_mask)
    output_soft_mask = UpSampling2D()(output_soft_mask)

    ## Output
    output_soft_mask = Conv2D(input_channels, (1, 1))(output_soft_mask)
    output_soft_mask = Conv2D(input_channels, (1, 1))(output_soft_mask)
    output_soft_mask = Activation('sigmoid')(output_soft_mask)

    # Attention: (1 + output_soft_mask) * output_trunk
    output = Lambda(lambda x: x + 1)(output_soft_mask)
    output = Multiply()([output, output_trunk])  #

    # Last Residual Block
    for i in range(p):
        output = residual_block(output)

    return output
  
def ResAttentionNet56(shape=(256, 256, 3), n_channels=64, n_classes=17,
                      dropout=0):


    input_ = Input(shape=shape)
    x = Conv2D(n_channels, (7, 7), strides=(2, 2), padding='same')(input_) # 112x112
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)  # 56x56

    x = residual_block(x, output_channels=n_channels * 4)  # 56x56
    x = attention_block(x, encoder_depth=3)  # bottleneck 7x7

    x = residual_block(x, output_channels=n_channels * 8, stride=2)  # 28x28
    x = attention_block(x, encoder_depth=2)  # bottleneck 7x7

    x = residual_block(x, output_channels=n_channels * 16, stride=2)  # 14x14
    x = attention_block(x, encoder_depth=1)  # bottleneck 7x7

    x = residual_block(x, output_channels=n_channels * 32, stride=2)  # 7x7
    x = residual_block(x, output_channels=n_channels * 32)
    x = residual_block(x, output_channels=n_channels * 32)

    pool_size = (x.get_shape()[1].value, x.get_shape()[2].value)
    x = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x)
    if dropout:
        x = Dropout(dropout)(x)
    output = Dense(n_classes, activation='sigmoid')(x)

    model = Model(input_, output)
    
    return model

def my_model(shape=(256,256,3)):
  input_ = Input(shape=shape)
  a1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(input_)

  x1 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(input_)
  x1 = Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')(x1) 
  x1 = BatchNormalization()(x1)
  x1 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x1)
  x1 = concatenate([a1, x1])

  a2 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(a1)
  x2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x1)
  x2 = Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu')(x2) 
  x2 = BatchNormalization()(x2)
  x2 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x2)
  x2 = concatenate([a2, x2])
  
  a3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(a2)
  x3 = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x2)
  x3 = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x3) 
  x3 = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(x3) 
  x3 = BatchNormalization()(x3)
  x3 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x3)
  x3 = concatenate([a3, x3])
  
  a4 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(a3)
  x4 = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x3)
  x4 = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x4) 
  x4 = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x4) 
  x4 = BatchNormalization()(x4)
  x4 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x4)
  x4 = concatenate([a4, x4])
  
  a5 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(a4)
  x5 = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x4)
  x5 = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x5) 
  x5 = Conv2D(512, (3, 3), strides=(1, 1), padding='same', activation='relu')(x5)
  x5 = BatchNormalization()(x5)
  x5 = Activation('relu')(x5)
  x5 = MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding='same')(x5)
  x5 = concatenate([a5, x5])
  
  pool_size = (x5.get_shape()[1].value, x5.get_shape()[2].value)
  x5 = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x5)
  x5 = Flatten()(x5)
  #x5 = Dropout(0.50)(x5)
  
  pool_size = (x4.get_shape()[1].value, x4.get_shape()[2].value)
  x4 = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x4)
  x4 = Flatten()(x4)
  #x4 = Dropout(0.50)(x4)
  
  pool_size = (x3.get_shape()[1].value, x3.get_shape()[2].value)
  x3 = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x3)
  x3 = Flatten()(x3)
  #x3 = Dropout(0.50)(x3)
  
  pool_size = (x2.get_shape()[1].value, x2.get_shape()[2].value)
  x2 = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x2)
  x2 = Flatten()(x2)
  #x2 = Dropout(0.50)(x2)
  
  pool_size = (x1.get_shape()[1].value, x1.get_shape()[2].value)
  x1 = AveragePooling2D(pool_size=pool_size, strides=(1, 1))(x1)
  x1 = Flatten()(x1)
  #x1 = Dropout(0.50)(x1)

  x = concatenate([x1, x2, x3, x4, x5],axis=-1)

  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)

  output = Dense(17, activation='sigmoid')(x)
  
  model = Model(input_, output)
  
    
  return model

In [23]:
model1 = my_model()
op = Adam(lr=0.0001)
model1.compile(loss='binary_crossentropy', optimizer=op, metrics=['accuracy'])
model1.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_27 (Conv2D)              (None, 256, 256, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_28 (Conv2D)              (None, 256, 256, 64) 36928       conv2d_27[0][0]                  
__________________________________________________________________________________________________
batch_normalization_11 (BatchNo (None, 256, 256, 64) 256         conv2d_28[0][0]                  
____________________________________________________________________________________________

In [24]:
data_augmentation = 0
if data_augmentation == 1:
  model1.fit_generator( data_generator.flow(train_set, train_labels_encoded, batch_size=32),
                                               epochs=35,
                                               steps_per_epoch=1600 // 32,
                                               verbose=2,
                                               validation_data=(test_set,test_labels_encoded),
                                               callbacks=CALLBACKS() )
else:
  model1.fit( train_set, train_labels, batch_size = 32, epochs = 35, 
                    validation_data=(test_set,test_labels), callbacks=CALLBACKS())


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Train on 1700 samples, validate on 400 samples
Epoch 1/35
1700/1700 [==============================] - 57s 34ms/step - loss: 0.3007 - acc: 0.8663 - val_loss: 0.2901 - val_acc: 0.8797
Epoch 2/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.2275 - acc: 0.8999 - val_loss: 0.3515 - val_acc: 0.8685
Epoch 3/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.1976 - acc: 0.9143 - val_loss: 0.2861 - val_acc: 0.8772
Epoch 4/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.1814 - acc: 0.9212 - val_loss: 0.2481 - val_acc: 0.8937
Epoch 5/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.1698 - acc: 0.9263 - val_loss: 0.2446 - val_acc: 0.9041
Epoch 6/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.1532 - acc: 0.9336 - val_loss: 0.1995 - val_acc: 0.9141
Epoch 7/35
1700/1700 [==============================] - 55s 32ms/step - loss: 0.1373 - acc: 0.9412 - val_loss: 0.2018 - val_acc: 0.9138
E

In [25]:
thresholding1(test_set, test_labels)

[0.27 0.43 0.59 0.44 0.48 0.1  0.1  0.21 0.55 0.12 0.56 0.71 0.1  0.4
 0.31 0.45 0.31]
0.04058823529411765
Accuracy is :: 0.8528214285714286
F1 Score is :: 0.9194991254436934
F2 Score is :: 0.9149417841684517
Precision row :: 0.9271964285714286
Recall row :: 0.9119285714285713
Precision column :: 0.9324684092493127
Recall column :: 0.8987238920530579
  Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.87      0.78      0.82       144
           2       0.87      0.83      0.85       131
           3       0.89      0.82      0.86       163
           4       1.00      1.00      1.00        24
           5       0.81      0.57      0.67        23
           6       1.00      1.00      1.00        24
           7       1.00      0.89      0.94        18
           8       0.90      0.89      0.89       175
           9       0.95      1.00      0.98        21
          10       0.92   